In [1]:
import numpy as np
import cv2

In [11]:
def nms(detections: np.ndarray, confidence_threshold: float=0.25, score_threshold: float=0.2, nms_threshold: float=0.45) -> list:
    """后处理

    Args:
        detections (np.ndarray):                检测到的数据 [25200, 85]
        confidence_threshold (float, optional): prediction[4] 是否有物体得分阈值. Defaults to 0.25.
        score_threshold (float, optional):      nms分类得分阈值. Defaults to 0.2.
        nms_threshold (float, optional):        非极大值抑制iou阈值. Defaults to 0.45.

    """
    boxes = []  # [[xmin, ymin, w, h]]
    class_ids = []
    confidences = []
    for prediction in detections:
        confidence = prediction[4].item()           # 是否有物体得分
        if confidence >= confidence_threshold:      # 是否有物体预支
            classes_scores = prediction[5:]         # 取出所有类别id
            class_id = np.argmax(classes_scores)    # 找到概率最大的id
            if (classes_scores[class_id] > .25):    # 最大概率必须大于 0.25
                confidences.append(confidence)      # 保存置信度(注意保存的是confidence，不是classes_scores[class_id]),类别id,box
                class_ids.append(class_id)
                # center_x, center_y, w, h
                x, y, w, h = prediction[0].item(), prediction[1].item(), prediction[2].item(), prediction[3].item()
                xmin = x - (w / 2)
                ymin = y - (h / 2)
                box = [xmin, ymin, w, h]
                boxes.append(box)

    # nms
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold, nms_threshold)
    # t3 = time.time()

    # 根据nms返回的id获取对应的类别,置信度,box
    detections = []
    for i in indexes:
        j = i.item()
        boxes[j][2] += boxes[j][0] # w -> xmax
        boxes[j][3] += boxes[j][1] # h -> ymax
        detections.append({"class_index": class_ids[j], "confidence": confidences[j], "box": boxes[j]})
    # t4 = time.time()

    # print((t2 - t1)*1000, (t3 - t2)*1000, (t4 - t3)*1000)
    # 16.954421997070312 0.0 0.0 主要时间花在了遍历所有的框上面

    return detections

In [12]:
def nms1(detections: np.ndarray, confidence_threshold: float=0.25, score_threshold: float=0.2, nms_threshold: float=0.45) -> np.ndarray:
    """后处理

    Args:
        detections (np.ndarray):                检测到的数据 [25200, 85]
        confidence_threshold (float, optional): prediction[4] 是否有物体得分阈值. Defaults to 0.25.
        score_threshold (float, optional):      nms分类得分阈值. Defaults to 0.2.
        nms_threshold (float, optional):        非极大值抑制iou阈值. Defaults to 0.45.

    Returns:
        np.ndarray:
            [
                [class_index, confidences, xmin, ymin, xmax, ymax], np.float32
                ...
            ]
    """
    # boxes = []  # [[xmin, ymin, w, h]]
    # class_ids = []
    # confidences = []
    # 循环25200次,很慢,大约12ms
    # for prediction in detections:
    #     confidence = prediction[4].item()           # 是否有物体得分
    #     if confidence >= confidence_threshold:      # 是否有物体预支
    #         classes_scores = prediction[5:]         # 取出所有类别id
    #         class_id = np.argmax(classes_scores)    # 找到概率最大的id
    #         if (classes_scores[class_id] > .25):    # 最大概率必须大于 0.25
    #             confidences.append(confidence)      # 保存置信度(注意保存的是confidence，不是classes_scores[class_id]),类别id,box
    #             class_ids.append(class_id)
    #             # center_x, center_y, w, h
    #             x, y, w, h = prediction[0].item(), prediction[1].item(), prediction[2].item(), prediction[3].item()
    #             xmin = x - (w / 2)
    #             ymin = y - (h / 2)
    #             box = [xmin, ymin, w, h]
    #             boxes.append(box)

    # 加速优化写法
    # 通过置信度过滤一部分框
    detections = detections[detections[:, 4] > confidence_threshold]
    # 位置坐标
    loc = detections[:, :4]
    # 置信度
    confidences = detections[:, 4]
    # 分类
    cls = detections[:, 5:]
    # 最大分类index
    max_cls_index = cls.argmax(axis=-1)
    # 最大分类score
    max_cls_score = cls.max(axis=-1)
    # 置信度
    confidences = confidences[max_cls_score > .25]
    # 类别index
    class_index = max_cls_index[max_cls_score > .25]
    # 位置
    boxes = loc[max_cls_score > .25]
    # [center_x, center_y, w, h] -> [x_min, y_min, w, h]
    boxes[:, 0] -= boxes[:, 2] / 2
    boxes[:, 1] -= boxes[:, 3] / 2

    # nms
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold, nms_threshold)

    # 根据nms返回的id获取对应的类别,置信度,box
    # detections = []
    # for i in indexes:
    #     j = i.item()
    #     boxes[j][2] += boxes[j][0] # w -> xmax
    #     boxes[j][3] += boxes[j][1] # h -> ymax
    #     detections.append({"class_index": class_index[j], "confidence": confidences[j], "box": boxes[j]})

    boxes[indexes, 2] += boxes[indexes, 0]  # w -> xmax
    boxes[indexes, 3] += boxes[indexes, 1]  # h -> ymax
    # [
    #   [class_index, confidences, xmin, ymin, xmax, ymax],
    #   ...
    # ]
    detections = np.concatenate((np.expand_dims(class_index[indexes], 1), np.expand_dims(confidences[indexes], 1), boxes[indexes]), axis=-1)

    return detections

In [13]:
x = np.random.normal(0, 1, (25200, 85))
x.shape

(25200, 85)

In [ ]:
%timeit nms(x, 0.25, 0.2, 0.45)

In [14]:
r = nms(x, 0.25, 0.2, 0.45)

1011 [  21 4043 6820 ... 3661  333 7888]


In [ ]:
%timeit nms1(x, 0.25, 0.2, 0.45)

In [15]:
r = nms1(x, 0.25, 0.2, 0.45)

1011 [  21 4043 6820 ... 3661  333 7888]


In [29]:
detections = x
detections.shape

(25200, 85)

In [30]:
# 通过置信度过滤一部分框
detections = detections[detections[:, 4] > 0.25]
detections.shape

(10013, 85)

In [31]:
# 置信度
confidences = detections[:, 4]
confidences.shape

(10013,)

In [32]:
# 位置坐标
loc = detections[:, :4]
# 分类
cls = detections[:, 5:]
loc.shape, cls.shape

((10013, 4), (10013, 80))

In [34]:
# 最大分类index
max_cls_index = cls.argmax(axis=-1)
# 最大分类score
max_cls_score = cls.max(axis=-1)
max_cls_index.shape, max_cls_score.shape

((10013,), (10013,))

In [41]:
# 置信度    (n,)
confidences = confidences[max_cls_score > .25]
# 类别index (n,)
class_index = max_cls_index[max_cls_score > .25]
# 位置      (n, 4)
boxes = loc[max_cls_score > .25]
confidences.shape, class_index.shape, boxes.shape

((10013,), (10013,), (10013, 4))

In [42]:
# [center_x, center_y, w, h] -> [x_min, y_min, w, h]
boxes[:, 0] -= boxes[:, 2] / 2
boxes[:, 1] -= boxes[:, 3] / 2

In [43]:
# nms
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.2, 0.45)
print(len(indexes), indexes)

1011 [  21 4043 6820 ... 3661  333 7888]


In [46]:
# 根据nms返回的id获取对应的类别,置信度,box
detections = []
for i in indexes:
    j = i.item()
    boxes[j][2] += boxes[j][0] # w -> xmax
    boxes[j][3] += boxes[j][1] # h -> ymax
    detections.append({"class_index": class_index[j], "confidence": confidences[j], "box": boxes[j]})
len(detections)

1011

In [48]:
print(class_index[indexes].shape)
print(confidences[indexes].shape)
print(boxes[indexes].shape)

(1011,)
(1011,)
(1011, 4)


In [59]:
detections = np.concatenate((np.expand_dims(class_index[indexes], 1), np.expand_dims(confidences[indexes], 1), boxes[indexes]), axis=-1)
detections.shape

(1011, 6)

In [64]:
print(np.all(detections[:, 0] == class_index[indexes]))
print(np.all(detections[:, 1] == confidences[indexes]))
print(np.all(detections[:, 2:] == boxes[indexes]))

True
True
True


dtype('float64')